# Template for Creating Sacrament Meeting Programs

## Uses MarkupPy to generate the HTML

In [14]:
#
# Sugar Land 2nd Ward HTML Program Writer
#
# Pete Slater
# June 2022

import hymndict # Titles and links to hymns
import htmlpy # Local copy of source code
from htmlpy import oneliner as e
import artlinks # Links to artwork
import sys
import datetime as dt

# Customized functions to create the HTML for standard agenda items

In [15]:
'''
Define functions for formatting each of the items that can appear in a program.
Build the program in the main script by calling the functions.

'''
def officer(role, officer, page):
    page.p(e.b(role+":")+" "+officer,align="center")

def pagetitle(unit, meeting, meetdate,page):
    page.h1(unit, align="center")
    page.h2(meeting, align="center")
    page.h2(meetdate, align="center")

def speaker(name, page):
    page.h2("Speaker", align="center")
    page.p(name,align="center")
    
def testimonies(page):
    page.h2("Bearing of Testimonies", align="center")

def music(number, description, page):
    page.h2(description,align="center")
    [hymntitle, hymnurl] = hymndict.hymns[number]
    page.p(e.a("#" + str(number)+", "+hymntitle, href=hymnurl),align="center")
    
def specialmusic(performers,page,title=None,accompanist=None):
    page.h2("Special Musical Number",align="center")
    if title != None:
        page.p(e.i(title),align="center")
    page.p(performers,align="center")
    if accompanist != None:
        page.p("Acc. by " + accompanist, align="center")
        
def thought(text, author, page):
    page.hr()
    page.p(text, align="center")
    page.p("- "+author,align="center")
 
# Announcements are passed as a list of text strings
def announcements(textlist,page):
    page.hr()
    page.h1("Announcements", align="center")
    for txt in textlist:
        page.p(txt,align="center")

# Custom section for Preparedness Corner, etc.
def custom(title, txtlist, linklist, page):
    page.hr()
    page.h1(title, align="center")
    if txtlist != None:
        for txt in txtlist:
            page.p(txt,align="center")
    if linklist != None:
        for link in linklist:
            page.p(e.a(link[0], href=link[1], target="_blank", rel="noreferrer noopener"),align="center")
        
# Place some links, passed as a list containing text and url
def links(linklist, page):
    page.hr()
    page.h1("Links", align="center")
    for link in linklist:
        page.p(e.a(link[0], href=link[1], target="_blank", rel="noreferrer noopener"),align="center")
        
# Print credits at the bottom of the page
def credit(editor, page):
#'''Print credits and datestamp ''' 
    today = dt.date.today()
    page.hr()
    text = today.strftime("Edited %B %d, %Y by ") + editor
    page.small(text,align="center")        
    

# Format the agenda items that are always present
Customize for each week with date and type of meeting, i.e. sacrament, fast and testimony, ward conference

In [16]:
Title = "Sugar Land Second Ward"
header = "Sacrament Meeting"
#header = "Fast and Testimony Meeting"
footer = ""
styles = ( 'layout.css', 'alt.css', 'images.css' )

page = htmlpy.page( )
page.init()
page.br( )
 
# Make sure it will look good on all devices
page.meta(name="viewport", content="width=device-width, initial-scale=1.0")

# Define the elements on the current week's programs here

pagetitle("Sugar Land 2nd Ward", "Sacrament Meeting", "April 9, 2023", page)

## Artwork
Pick an artwork from the defined list of links in artlinks.py and enter the number here

In [17]:
# Place an artwork from the imported dictionary of links
#page.p(e.img(width=299, height=300*0.8, src=artlinks.art[2]), align="center")
page.p(e.img(style="max-width:50%;height:auto;", src=artlinks.art['Easter1']), align="center")

KeyError: 'Easter1'

## Officers

In [ ]:
officer("Presiding","Bishop Joey Powell", page)
officer("Conducting","Bishop Joey Powell", page)
#officer("Conducting","Brother Dale Crosby", page)
#officer("Conducting","Brother Jared Draney", page)

## Opening and sacrament hymns

In [ ]:
music(198,"Opening Hymn",page)
music(175,"Sacrament Hymn", page) 

## Variable part of the program

### If it's fast and testimony meeting, uncomment this section and comment out the sacrament meeting items

In [ ]:
# Uncomment this line for fast and testimony meeting
#testimonies(page)

### For a sacrament meeting or ward conference, comment out the testimonies and use these sections as needed

In [ ]:
# Use lines like these to form the speaking part of the program

# Special code for the Primary and Christmas Programs
#txtlist = (
#    'Words and Music Celebrating the Birth of Jesus Christ',
#    'Program under the direction of Mindy Draney',
#    'Program written by Becky Slater',
#    'Congregation to join in singing when directed.'
#    )
#linklist = None

#custom("Primary Program", txtlist,linklist, page)

#music(301,"Intermediate Hymn",page)
specialmusic("Primary Children",page, title="Jesus Has Risen", accompanist="Ting Chen")
speaker("Melinda Draney", page)

specialmusic("Dale Jeffrey, Veniece & Jerry Griffin, and Maria Boyer",page, title="In the Garden")
speaker("Leonardo Garcia", page)

specialmusic("Walk-Up Choir",page, title="Christ The Lord is Risen Today")

#speaker("Reed Bradford", page)
#speaker("Ben Powell", page)

### Closing hymn

In [ ]:
music(199,"Closing Hymn",page)

## End matter - thought, announcements, links

In [ ]:
# Spiritual thought
quote = ('I discovered there was no handle on the outside of the door the Savior was knocking on. The person on the inside needed to open the door. Now I know who is on the other side of the door. We are! The Savior is knocking, and we all have to open the door and invite Him to come into our lives.')
author = 'Elder O. Vincent Haleck Of the Seventy Oct. 2013'
thought(quote, author, page)

# Make a list of announcements, then post to the page
txtlist = (
        "APRIL 15 : African American Heritage Project: Jones Dickson Cemetary 8:30 AM - 11:00 AM",
                    "We invite Stake youth and adults to help wrap up the land-clearing phase of this cemetery reclamation and restoration project.",
                    "Lawn mowers, weed-eaters and leaf rakes will be needed for this endeavor.",
                    "Please email meliaboyer@justserve.org if you can bring any of the needed equipment.",
                    "Also please register to participate in the project on Nov JustServe.org so that we can better log volunteer hours for this project.",
        "APRIL 29 : African American Heritage Project: Jones Dickson Cemetary 8:30 AM - 11:00 AM",
                    "We invite Stake youth and adults to help wrap up the land-clearing phase of this cemetery reclamation and restoration project.",
                    "Lawn mowers, weed-eaters and leaf rakes will be needed for this endeavor.",
                    "Please email meliaboyer@justserve.org if you can bring any of the needed equipment.",
                    "Also please register to participate in the project on Nov JustServe.org so that we can better log volunteer hours for this project.",
        "APRIL 29 : Multi-Stake Youth Simply Prom  7:00 PM - 11:00 PM.",        
        "Send announcements to bvl2clerk@gmail.com"
           )
announcements(txtlist, page)

### Preparedness Corner - under development

In [ ]:
#txtlist = (
#    'Make sure you have water: one gallon per person per day. A family of four would need 12 gallons for 3 days.',
#    ' '
#    )
#linklist = {
#    ("Learn more online","https://www.churchofjesuschrist.org/study/manual/gospel-topics/food-storage?lang=eng")
#}
#custom("Preparedness Corner", txtlist,linklist, page)

In [ ]:
# Make a list of links, then post
linklist = (
        ("Sing up for African American Heritage Project: Jones Dickson Cemetary April 15th & 29th @ 8:30 AM - 11:00 AM","https://www.justserve.org/projects/c52cbafd-414d-4366-b7ef-6607246ea80d/african-american-heritage-project%3A-jones-dickson-cemetery?shiftId=328fd664-ac07-4f83-bc1c-777aa157d274"),
        ("Sign up to feed the missionaries","https://www.signupgenius.com/go/10c0e4baca82ea7fb6-dinner2"),
        ("Sign up to prepare a meal with Fort Bend Family Promise","https://www.signupgenius.com/go/10c0e4baca82ea7fb6-helping4#"),
        ("Help at secondmile.org","https://secondmile.org"), 
        )
links(linklist, page)
credit("Pete Slater and Andy Chen", page)

# Write out the file to Index.html, which must be posted online

In [ ]:
print (page)# -*- coding: utf-8 -*

original_stdout = sys.stdout
with open('Index.html', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(page)
    sys.stdout = original_stdout # Reset the standard output to its original value
"""
End of script
"""